<a href="https://colab.research.google.com/github/daspartho/learn-pytorch/blob/main/datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
from torch import nn

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'